# Telegram Chatbot Backed by KGQA with Lambda and API Gateway

References:
- [Building Your First Serverless Telegram Bot with AWS Lambda](https://iamondemand.com/blog/building-your-first-serverless-telegram-bot/)
- [github:lesterchan/telegram-bot ](https://github.com/lesterchan/telegram-bot)
- [Simple Telegram Bot with Python and AWS Lambda](https://levelup.gitconnected.com/simple-telegram-bot-with-python-and-aws-lambda-5eab1066b466)
- [API GatewayとLambda(Python)でLINE BOT(Messaging API)開発](https://qiita.com/w2or3w/items/1b80bfbae59fe19e2015)
- [API Gateway + LambdaでLINE Bot開発](https://xp-cloud.jp/blog/2019/06/24/5560/)
- [Serverless github bot with AWS Lambda and API Gateway](https://kalinchernev.github.io/serverless-github-bot-aws-lambda-api-gateway-nodejs)
- [Writing a Serverless Slack Bot](https://messagemedia.com/au/blog/writing-a-serverless-slack-bot/)
- [Build Chatbots using Serverless Bot Framework with Salesforce Integration](https://aws.amazon.com/blogs/architecture/build-chatbots-using-serverless-bot-framework-with-salesforce-integration/)
- [How to Deploy a Telegram Bot with Flask, pyTelegramBotAPI, Gunicorn and PostgreSQL to Heroku](https://medium.com/tech-insights/how-to-deploy-a-telegram-bot-with-flask-pytelegrambotapi-gunicorn-and-postgresql-to-heroku-19d87959a65)
- [I built a serverless Telegram bot over the weekend. Here’s what I learned.](https://www.freecodecamp.org/news/how-to-build-a-server-less-telegram-bot-227f842f4706/)

## Part 1: End-to-end Invocation

### 1. Define and retrieve Neptune and inference endpoints

We write codes of converting a question to an answer in this part.

Environment variables:

In [3]:
import boto3
neptune_db_cluster_identifier = 'kg-neptune'
nlu_endpoint_name_contains = 'qa-model-from-registry-ep'

In [4]:
neptune = boto3.client('neptune')
response = neptune.describe_db_clusters(DBClusterIdentifier=neptune_db_cluster_identifier)
neptue_endpoint_desc = response['DBClusters'][0]
neptue_endpoint_desc

{'AllocatedStorage': 1,
 'AvailabilityZones': ['us-east-1b', 'us-east-1a', 'us-east-1c'],
 'BackupRetentionPeriod': 1,
 'DBClusterIdentifier': 'kg-neptune',
 'DBClusterParameterGroup': 'default.neptune1',
 'DBSubnetGroup': 'default',
 'Status': 'available',
 'EarliestRestorableTime': datetime.datetime(2021, 9, 29, 9, 56, 9, 536000, tzinfo=tzlocal()),
 'Endpoint': 'kg-neptune.cluster-c2ycbhkszo5s.us-east-1.neptune.amazonaws.com',
 'ReaderEndpoint': 'kg-neptune.cluster-ro-c2ycbhkszo5s.us-east-1.neptune.amazonaws.com',
 'MultiAZ': False,
 'Engine': 'neptune',
 'EngineVersion': '1.0.5.0',
 'LatestRestorableTime': datetime.datetime(2021, 9, 30, 10, 25, 37, 733000, tzinfo=tzlocal()),
 'Port': 8182,
 'MasterUsername': 'admin',
 'PreferredBackupWindow': '09:47-10:17',
 'PreferredMaintenanceWindow': 'fri:05:05-fri:05:35',
 'ReadReplicaIdentifiers': [],
 'DBClusterMembers': [{'DBInstanceIdentifier': 'kg-neptune-instance-1',
   'IsClusterWriter': True,
   'DBClusterParameterGroupStatus': 'in-sync

In [5]:
sm = boto3.client('sagemaker')
response = sm.list_endpoints(
    NameContains=nlu_endpoint_name_contains
)
nlu_endpoint_desc = response['Endpoints'][0]
nlu_endpoint_desc

{'EndpointName': 'qa-model-from-registry-ep',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:093729152554:endpoint/qa-model-from-registry-ep',
 'CreationTime': datetime.datetime(2021, 9, 28, 1, 54, 31, 267000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 9, 28, 2, 2, 51, 936000, tzinfo=tzlocal()),
 'EndpointStatus': 'InService'}

In [6]:
neptune_endpoint = neptue_endpoint_desc['Endpoint']
neptune_endpoint_port = neptue_endpoint_desc['Port']
nlu_endpoint_name = nlu_endpoint_desc['EndpointName']

### 2. Dump query templates

Copy query templates to chatbot resources. Query templates are copied from QA data generation notebook.

In [7]:
query_templates = {
    'ask_alumni': "g.V().has('学校', 'name', '{}').inE().hasLabel('毕业院校').outV().values('name').toList()",
    'ask_school': "g.V().has('人物','name','{}').out('毕业院校').values('name').next()",
    'ask_books': "g.V().has('人物', 'name', '{}').inE().hasLabel('作者').outV().values('name').toList()", 
    'ask_author': "g.V().has('图书作品','name','{}').out('作者').values('name').next()",
    'ask_wife': "g.V().has('人物','name','{}').out('妻子').values('name').next()",
    'ask_husband': "g.V().has('人物','name','{}').out('丈夫').values('name').next()",
    'ask_films': "g.V().has('人物', 'name', '{}').inE().hasLabel('导演').outV().values('name').toList()",
    'ask_director': "g.V().has('影视作品','name','{}').out('导演').values('name').next()",
    'ask_nationality': "g.V().has('人物','name','{}').out('国籍').values('name').next()"
}

In [8]:
import json
with open('chatbot_resource/query_templates.json', 'w') as f:
    json.dump(query_templates, f, ensure_ascii=False)

### 3. Parse questions and query Neptune

You can go ahead to part 2 without runing the remaining section in section 3.

Function for running graph query

In [ ]:
!pip install gremlinpython

In [200]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

def query_neptune(expr, neptune_endpoint, port):
    graph = Graph()
    if port == 80 or port == '80': # use unencrypted web socket if port is an http port
        neptune_web_socket = f"ws://{neptune_endpoint}:{port}/gremlin"
    else:
        neptune_web_socket = f"wss://{neptune_endpoint}:{port}/gremlin"
    remoteConn = DriverRemoteConnection(neptune_web_socket,'g')
    g = graph.traversal().withRemote(remoteConn)
    result = eval(expr)
    remoteConn.close()
    return result

Function for converting question to intention and values. 

In [29]:
import json
import sagemaker
from sagemaker.pytorch.model import PyTorchPredictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

def parse_questions(questions, nlu_endpoint_name):
    '''
    Args:
        questions (list(str)): A list of natural language questions
    '''
    sess = sagemaker.Session()
    predictor = PyTorchPredictor(
        endpoint_name=nlu_endpoint_name,
        sagemaker_session=sess,
        serializer=CSVSerializer(),
        deserializer=JSONDeserializer(),
    )
    predicted = predictor.predict(questions)
    return predicted['text'], predicted['intentions'], predicted['slot_labels']

In [49]:
def extract_slot_values(question, seq_label):
    assert len(question) == len(seq_label), f"question {question} should have the same \
length with sequence label {seq_label} ({len(question)} != {len(seq_label)})"
    value_buf = ''
    slot_buf = ''
    values = []
    slots = []
    for i, l in enumerate(seq_label):
        if l.startswith('B'):
            if value_buf != '':
                values.append(value_buf)
                slots.append(slot_buf)
            slot_buf = l[2:] # extract label part from B_label
            value_buf = question[i]
        elif l.startswith('I'):
            value_buf += question[i]
        elif l.startswith('O'):
            if value_buf != '':
                values.append(value_buf)
                slots.append(slot_buf)
            value_buf = ''
            slot_buf = ''  
    return slots, values

Function for generating graph query:

In [53]:
def generate_graph_query(intent, slots, values, query_templates):
    if intent not in query_templates.keys():
        raise Exception(f"Query templates does not have a template for {intent}")
    template = query_templates[intent]
    query = template.format(*values)
    return query

In [89]:
question = '张艺谋导演了哪些电影'

In [54]:
_, intentions, slot_labels = parse_questions([question], nlu_endpoint_name)
print('Intentions:')
print(intentions)
print('Slot labels:')
print(slot_labels)

Intentions:
['ask_films']
Slot labels:
[['B_name', 'I_name', 'I_name', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [55]:
slots, values = extract_slot_values(question, slot_labels[0])
slots, values

(['name'], ['张艺谋'])

In [57]:
query = generate_graph_query(intentions[0], slots, values, query_templates)
query

"g.V().has('人物', 'name', '张艺谋').inE().hasLabel('导演').outV().values('name').toList()"

To run the query event loop in jupyter notebook event look, run following cell:

In [60]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


Use this command to check connectivity and status of your Neptune endpoint:

In [70]:
!curl database-2.cluster-ro-c2ycbhkszo5s.us-east-1.neptune.amazonaws.com:8182/status

curl: (28) Failed to connect to database-2.cluster-ro-c2ycbhkszo5s.us-east-1.neptune.amazonaws.com port 8182: Connection timed out


In [71]:
!curl alb-neptune-test-62758122.us-east-1.elb.amazonaws.com/status

{"status":"healthy","startTime":"Fri Aug 20 04:52:57 UTC 2021","dbEngineVersion":"1.0.4.2.R5","role":"writer","gremlin":{"version":"tinkerpop-3.4.10"},"sparql":{"version":"sparql-1.1"},"labMode":{"NeptuneML":"disabled","ObjectIndex":"disabled","DFEQueryEngine":"disabled","ReadWriteConflictDetection":"enabled"},"resultCache":{"status":"Disabled"}}

In [79]:
query_neptune(query, 'alb-neptune-test-62758122.us-east-1.elb.amazonaws.com', 80)

['阳光灿烂的日子',
 '三枪',
 '一个陌生女人的来信',
 '2046',
 '三枪拍案惊奇',
 '万里长城',
 '山楂树之恋',
 '金陵十三钗',
 '印象·刘三姐',
 '影子武士',
 '秋菊打官司',
 '太阳照常升起',
 '让子弹飞',
 '王朝的女人·杨贵妃',
 '建国大业',
 '大红灯笼高高挂',
 '英雄',
 '我的父亲母亲',
 '妻妾成群',
 '菊豆',
 '左耳',
 '山乡书记',
 '北京人在纽约',
 '对话·寓言2047',
 '十面埋伏',
 '长城',
 '习近平',
 '栀子花开']

In [175]:
def question2answer(question, query_templates, nlu_endpoint_name, neptune_endpoint, neptune_endpoint_port):
    _, intentions, slot_labels = parse_questions([question], nlu_endpoint_name)
    print(f"Intention: {intentions[0]}")
    slots, values = extract_slot_values(question, slot_labels[0])
    print(f"Slot labels: {slots},{values}")
    query = generate_graph_query(intentions[0], slots, values, query_templates)
    print(f"Query: {query}")
    try:
        query_result = query_neptune(query, neptune_endpoint, neptune_endpoint_port)
    except Exception as e:
        print(e)
        query_result = '我不知道'
    return query_result

In [209]:
question = '异界之再战风云是谁的作品'

In [210]:
question2answer(question, query_templates, nlu_endpoint_name, 'alb-neptune-test-62758122.us-east-1.elb.amazonaws.com', 80)

Intention: ask_author
Slot labels: ['book'],['异界之再战风云']
Query: g.V().has('图书作品','name','异界之再战风云').out('作者').values('name').next()


'品味人生'

## Part 2: Deploy an Invocation Lambda Function

This architecture looks like this:
![Access Neptune from Lambda](https://aws-samples.github.io/aws-dbs-refarch-graph/src/accessing-from-aws-lambda/lambda-neptune.png)

> **Walkthrough of the Architecture** </br>
> - In this architecture your Neptune cluster is run in at least two subnets in two Availability Zones, with each subnet in a different Availability Zone. By distributing your cluster instances across at least two Availability Zones, you help ensure that there are instances available in your DB cluster in the unlikely event of an Availability Zone failure.
> - Neptune’s VPC security group is configured to allow access from the AWS Lambda security group on the Neptune cluster’s port.
> - AWS Lambda is configured to access resources in your VPC. Doing so allows Lambda to create elastic network interfaces (ENIs) that enable your function to connect securely to Neptune.</br>
> - The Lambda VPC configuration information includes at least 2 private subnets, allowing Lambda to run in high availability mode.
> - The VPC security group that Lambda uses is permitted to access Neptune via an inbound rule on the Neptune VPC security group.
> - Code running in your Lambda function uses a Gremlin or SPARQL client to submit queries to the Neptune cluster’s cluster, reader and/or instance endpoints.
> - API Gateway exposes API operations that accept client requests and execute your backend Lambda functions.


References:
- [Accessing Amazon Neptune from AWS Lambda Functions](https://aws-samples.github.io/aws-dbs-refarch-graph/src/accessing-from-aws-lambda/)
- Role setting: [Configuring a Lambda function to access resources in a VPC](https://docs.aws.amazon.com/lambda/latest/dg/configuration-vpc.html)

### 1. Gather Neptune VPC information

Get VPC ID of neptune db:

In [9]:
respones = neptune.describe_db_instances(
        Filters=[
            {
                'Name': 'db-cluster-id',
                'Values': [neptune_db_cluster_identifier]
            },
            {
                'Name': 'engine',
                'Values': ['neptune']
            }
        ]
    )
db_instances = respones['DBInstances']

In [10]:
neptune_security_groups = neptue_endpoint_desc['VpcSecurityGroups']
neptune_secutiry_group_ids = [security_group['VpcSecurityGroupId'] for security_group in neptune_security_groups]
neptune_secutiry_group_ids

['sg-dc24d3db']

In [11]:
first_neptune_instance_subnets = db_instances[0]['DBSubnetGroup']['Subnets']
neptune_subnet_ids = [subnet['SubnetIdentifier'] for subnet in first_neptune_instance_subnets]
neptune_subnet_ids

['subnet-fccf40cd',
 'subnet-f8676fb5',
 'subnet-2f611949',
 'subnet-a0d0e2ae',
 'subnet-94084eb5',
 'subnet-2b094c74']

In [12]:
neptune_instance_vpc_id = db_instances[0]['DBSubnetGroup']['VpcId']
neptune_instance_vpc_id

'vpc-851683f8'

### 2. Create an IAM role for lambda function

- [AWS Lambda execution role](https://docs.aws.amazon.com/lambda/latest/dg/lambda-intro-execution-role.html)
- [Configuring a Lambda function to access resources in a VPC](https://docs.aws.amazon.com/lambda/latest/dg/configuration-vpc.html)

In [13]:
iam_role_lambda_name = 'LambdaAccessNeptuneRole'

In [15]:
from botocore.exceptions import ClientError
import json

iam = boto3.client("iam")

In [14]:
assume_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
#         {
#             "Effect": "Allow", 
#             "Principal": {"Service": "events.amazonaws.com"}, 
#             "Action": "sts:AssumeRole"
#         },
        {
          "Effect": "Allow",
          "Principal": {"Service": "lambda.amazonaws.com"},
          "Action": "sts:AssumeRole"
        }
    ],
}

try:
    iam_role_lambda = iam.create_role(
        RoleName=iam_role_lambda_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description="Lambda Role to Interact with Neptune VPC",
    )
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

Role already exists


In [16]:
role_lambda = iam.get_role(RoleName=iam_role_lambda_name)
role_lambda_arn = role_lambda["Role"]["Arn"]

Attach AWS Managed Policy to the role to allow it to access a resource within a VPC - create, describe, delete network interfaces and write permissions to CloudWatch Logs.

In [263]:
response = iam.attach_role_policy(
    RoleName=iam_role_lambda_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaVPCAccessExecutionRole'
)

### 3. Create a security group for the Lambda function. 

This security group configures AWS Lambda to access resources in your VPC. Doing so allows Lambda to create elastic network interfaces (ENIs) that enable your function to connect securely to Neptune.

- [Configuring a Lambda function to access resources in a VPC](https://docs.aws.amazon.com/lambda/latest/dg/configuration-vpc.html)

In [17]:
lambda_security_group_name = 'lambda-kg-neptune-sg'
lambda_security_group_description = 'Allow Lambda function to access resources in a VPC'

In [255]:
# to delete an existing security group
# !aws ec2 delete-security-group --group-name $lambda_security_group_name

In [18]:
import boto3
from botocore.exceptions import ClientError

ec2 = boto3.client('ec2')

try:
    response = ec2.create_security_group(GroupName=lambda_security_group_name,
                                         Description=lambda_security_group_description,
                                         VpcId=neptune_instance_vpc_id)
    lambda_security_group_id = response['GroupId']
    print('Security Group Created %s in vpc %s.' % (lambda_security_group_id, neptune_instance_vpc_id))

    data = ec2.authorize_security_group_ingress(
        GroupId=lambda_security_group_id,
        IpPermissions=[
            {'IpProtocol': 'tcp',
             'FromPort': 80,
             'ToPort': 80,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
            {'IpProtocol': 'tcp',
             'FromPort': 22,
             'ToPort': 22,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
        ])
    print('Ingress Successfully Set %s' % data)
except ClientError as e:
    print(e)
    response = ec2.describe_security_groups(
        Filters=[
          {
              'Name': 'group-name',
              'Values': [
                  lambda_security_group_name
              ]
          },
      ]
    )
    lambda_security_group_id = response['SecurityGroups'][0]['GroupId']
lambda_security_group_id

An error occurred (InvalidGroup.Duplicate) when calling the CreateSecurityGroup operation: The security group 'lambda-kg-neptune-sg' already exists for VPC 'vpc-851683f8'


'sg-00a42a19c11b4b31a'

### 4. Allow Lambda Access Neptune

Also, we need to configure Neptune’s VPC security group to allow access from the AWS Lambda security group on the Neptune cluster’s port. 

In the PortRange text box, enter 8182, the default port value for a Neptune DB instance. Then enter choose a security group name in the Source text box.


References:
- [Use AWS Lambda functions with Amazon Neptune](https://aws.amazon.com/blogs/database/use-aws-lambda-functions-with-amazon-neptune/)
- [Creating a Security Group to Provide Access to a Neptune DB Instance in a VPC](https://docs.aws.amazon.com/neptune/latest/userguide/security-vpc-security-group.html)

In [269]:
try:
    for neptune_secutiry_group_id in neptune_secutiry_group_ids:
        ec2.authorize_security_group_ingress(
            GroupId=neptune_secutiry_group_id,
            SourceSecurityGroupName=lambda_security_group_name, # allow incoming connection from lambda's security group
        )
        print('Ingress Successfully Set %s' % data)
except ClientError as e:
    print(e)

Ingress Successfully Set {'Return': True, 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-005ebe058cc8f7fc3', 'GroupId': 'sg-00a42a19c11b4b31a', 'GroupOwnerId': '093729152554', 'IsEgress': False, 'IpProtocol': 'tcp', 'FromPort': 80, 'ToPort': 80, 'CidrIpv4': '0.0.0.0/0'}, {'SecurityGroupRuleId': 'sgr-0334230d3604590ff', 'GroupId': 'sg-00a42a19c11b4b31a', 'GroupOwnerId': '093729152554', 'IsEgress': False, 'IpProtocol': 'tcp', 'FromPort': 22, 'ToPort': 22, 'CidrIpv4': '0.0.0.0/0'}], 'ResponseMetadata': {'RequestId': '0712862b-eebb-4440-a4b8-11ecf796a9ca', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0712862b-eebb-4440-a4b8-11ecf796a9ca', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '1124', 'date': 'Wed, 29 Sep 2021 10:08:56 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


### 5. Create a Lambda function for to query Neptune DB:

- Check docs of `create_function`: [create_function](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda.html#Lambda.Client.create_function)
- An example of Lambda function accessing Neptune: [Lambda Functions Examples Python](https://docs.amazonaws.cn/en_us/neptune/latest/userguide/lambda-functions-examples.html#lambda-functions-examples-python)

Package source code and resources to a zip file:

> `-j` option do the following:</br>
Store just the name of a saved file (junk the path), and do not store directory 
names. By default, zip will store the full path (relative to the current 
directory).

We need to package dependencies manually, as specified here: [Deployment package with dependencies](https://docs.aws.amazon.com/lambda/latest/dg/python-package.html#python-package-create-package-with-dependency)
1. Install backoff and gremlin_python library to a new package directory

In [ ]:
!pip install --target ./package backoff gremlinpython tornado sagemaker numpy

2. Create a deployment package with the installed library at the root.

In [ ]:
%%bash
cd package
zip -r ../my-deployment-package.zip .

3. Add the lambda_function.py file to the root of the zip file. 

In [80]:
%%bash
zip -g -j my-deployment-package.zip chatbot_resource/*

updating: lambda_script.py (deflated 68%)
updating: query_templates.json (deflated 68%)


4. Upload it to S3

In [87]:
import sagemaker
default_bucket = sagemaker.Session().default_bucket()
lambda_function_script_key = 'lambda_scripts/query_neptune.zip'
!aws s3 cp my-deployment-package.zip s3://$default_bucket/$lambda_function_script_key

upload: ./my-deployment-package.zip to s3://sagemaker-us-east-1-093729152554/lambda_scripts/query_neptune.zip


Export credentials to environment in the same image console (or do it in a cell):

```bash
export AWS_ACCESS_KEY_ID=XXXXXXXXXXXXXXX
export AWS_SECRET_ACCESS_KEY=XXXXXXXXXXXXXXX
export AWS_REGION=us-east-1
export AWS_SESSION_TOKEN=XXXXXXXXXXXXXXX
```

In [292]:
sts = boto3.client('sts')
credentials = sts.get_session_token()['Credentials']

In [288]:
!export AWS_ACCESS_KEY_ID=
!export AWS_SECRET_ACCESS_KEY=credentials['SecretAccessKey']
!export AWS_SESSION_TOKEN=credentials['SessionToken']
!export AWS_REGION='us-east-1'

In [293]:
aws_access_key = credentials['AccessKeyId']
aws_secret_access_key = credentials['SecretAccessKey']
aws_session_token = credentials['SessionToken']
aws_region = 'us-east-1'

In [25]:
lmd = boto3.client('lambda')

In [22]:
function_name = 'query-kg-neptune'
# To connect to a VPC, the function execution role must have the following permissions
#    ec2:CreateNetworkInterface
#    ec2:DescribeNetworkInterfaces
#    ec2:DeleteNetworkInterface
function_exec_role = role_lambda_arn
# The name of the method within your code that Lambda calls to execute your function. 
# The format includes the file name. It can also include namespaces and other qualifiers, 
# depending on the runtime. 
function_handler = 'lambda_script.lambda_handler'
# The base64-encoded contents of the deployment package. Amazon Web Services SDK and Amazon 
# Web Services CLI clients handle the encoding for you.
# !apt update
# !apt install zip
!zip -j chatbot_resource.zip chatbot_resource/*
function_zip_bytes = open('chatbot_resource.zip', 'rb').read()

updating: lambda_script.py (deflated 68%)
updating: query_templates.json (deflated 68%)
  adding: __init__.py (deflated 18%)


In [23]:
# use this command to delete a previously created function
!aws lambda delete-function --function-name $function_name

In [26]:
import os

lmd.create_function(
    FunctionName=function_name,
    Runtime='python3.8',
    Role=function_exec_role,
    Handler=function_handler,
    Code={
#         'S3Bucket': default_bucket,
#         'S3Key': lambda_function_script_key,
        'ZipFile': function_zip_bytes
    },
    Description='query neptune db',
    Timeout=100,    # in seconds
    MemorySize=256, # in MB
    Publish=True,
    VpcConfig={
        'SubnetIds': neptune_subnet_ids[:3], # for convenience, using the same set of subnet ids as neptune subnets
        'SecurityGroupIds': [lambda_security_group_id] 
    },
    PackageType='Zip',
    Environment={
        'Variables': {
#             'AWS_ACCESS_KEY_ID': aws_access_key,
#             'AWS_SECRET_ACCESS_KEY': aws_secret_access_key,
#             'AWS_REGION': aws_region,
#             'AWS_SESSION_TOKEN': aws_session_token,
            'USE_IAM': 'false',
            'neptuneEndpoint': neptune_endpoint,
            'neptunePort': str(neptune_endpoint_port),
            'nluEndpoint': nlu_endpoint_name
        }
    }
)

{'ResponseMetadata': {'RequestId': 'd219d037-6af2-4fd8-8093-9e9a02fda344',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Thu, 30 Sep 2021 10:32:47 GMT',
   'content-type': 'application/json',
   'content-length': '1297',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd219d037-6af2-4fd8-8093-9e9a02fda344'},
  'RetryAttempts': 0},
 'FunctionName': 'query-kg-neptune',
 'FunctionArn': 'arn:aws:lambda:us-east-1:093729152554:function:query-kg-neptune',
 'Runtime': 'python3.8',
 'Role': 'arn:aws:iam::093729152554:role/LambdaAccessNeptuneRole',
 'Handler': 'lambda_script.lambda_handler',
 'CodeSize': 3229,
 'Description': 'query neptune db',
 'Timeout': 100,
 'MemorySize': 256,
 'LastModified': '2021-09-30T10:32:46.321+0000',
 'CodeSha256': 'Imihs1rty3vVL7KBI+s3zLE0DcdPVEdDhZ/tzCBJskA=',
 'Version': '12',
 'VpcConfig': {'SubnetIds': ['subnet-2f611949',
   'subnet-f8676fb5',
   'subnet-fccf40cd'],
  'SecurityGroupIds': ['sg-00a42a19c11b4b31a'],
  'VpcId': 'vpc-851683f8'},
 'Enviro

In [76]:
lmd.update_function_code(
    FunctionName=function_name,
    S3Bucket=default_bucket,
    S3Key=lambda_function_script_key,
    Publish=True,
)

{'ResponseMetadata': {'RequestId': '4b6bd986-75ca-4cc3-a142-a36e76bafb72',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 30 Sep 2021 04:14:35 GMT',
   'content-type': 'application/json',
   'content-length': '1274',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4b6bd986-75ca-4cc3-a142-a36e76bafb72'},
  'RetryAttempts': 1},
 'FunctionName': 'query-kg-neptune',
 'FunctionArn': 'arn:aws:lambda:us-east-1:093729152554:function:query-kg-neptune:7',
 'Runtime': 'python3.9',
 'Role': 'arn:aws:iam::093729152554:role/LambdaAccessNeptuneRole',
 'Handler': 'lambda_script.lambda_handler',
 'CodeSize': 3941576,
 'Description': 'query neptune db',
 'Timeout': 100,
 'MemorySize': 256,
 'LastModified': '2021-09-30T04:14:34.900+0000',
 'CodeSha256': 'J1J8c2olYjuASoeQWg745zPTRXEGmc7jrRtmDdv0fzc=',
 'Version': '7',
 'VpcConfig': {'SubnetIds': ['subnet-2f611949',
   'subnet-f8676fb5',
   'subnet-fccf40cd'],
  'SecurityGroupIds': ['sg-00a42a19c11b4b31a'],
  'VpcId': 'vpc-851683f8'},
 'En

In [91]:
import numpy

In [92]:
numpy.__version__

'1.20.3'

In [99]:
with open('chatbot_resource/query_templates.json') as f:
    t = json.load(f)

In [97]:
t

{'ask_alumni': "g.V().has('学校', 'name', '{}').inE().hasLabel('毕业院校').outV().values('name').toList()",
 'ask_school': "g.V().has('人物','name','{}').out('毕业院校').values('name').next()",
 'ask_books': "g.V().has('人物', 'name', '{}').inE().hasLabel('作者').outV().values('name').toList()",
 'ask_author': "g.V().has('图书作品','name','{}').out('作者').values('name').next()",
 'ask_wife': "g.V().has('人物','name','{}').out('妻子').values('name').next()",
 'ask_husband': "g.V().has('人物','name','{}').out('丈夫').values('name').next()",
 'ask_films': "g.V().has('人物', 'name', '{}').inE().hasLabel('导演').outV().values('name').toList()",
 'ask_director': "g.V().has('影视作品','name','{}').out('导演').values('name').next()",
 'ask_nationality': "g.V().has('人物','name','{}').out('国籍').values('name').next()"}